In [ ]:
from Crypto.Cipher import AES
from Crypto.Random import get_random_bytes
import base64
import pandas as pd
import numpy as np
from sklearn.ensemble import IsolationForest
import matplotlib.pyplot as plt
import seaborn as sns

# Helper functions for AES encryption and decryption
def pad(data):
    """ Pad data to be multiple of 16 bytes for AES encryption """
    return data + b' ' * (16 - len(data) % 16)

def encrypt(data, key):
    """ Encrypts data using AES encryption """
    cipher = AES.new(key, AES.MODE_ECB)
    return base64.b64encode(cipher.encrypt(pad(data)))

def decrypt(encrypted_data, key):
    """ Decrypts data using AES decryption """
    cipher = AES.new(key, AES.MODE_ECB)
    return cipher.decrypt(base64.b64decode(encrypted_data)).strip()

# Generate a random AES key for encryption
key = get_random_bytes(16)

# Sample Data: Simulating EV charging behavior dataset for malicious detection
np.random.seed(42)
data = {
    'userId': np.random.randint(1, 50, 100),
    'sessionId': range(1, 101),
    'reservationCount': np.random.randint(1, 5, 100),
    'actualUsage': np.random.choice([0, 1], size=100, p=[0.3, 0.7]),
    'cancellations': np.random.randint(0, 5, 100),
    'chargeTimeHrs': np.random.uniform(0, 10, 100),
}
station_data = pd.DataFrame(data)

# Simulate malicious behavior by injecting anomalies
malicious_users = np.random.choice(station_data.index, size=10, replace=False)
station_data.loc[malicious_users, 'reservationCount'] = np.random.randint(5, 10, size=10)
station_data.loc[malicious_users, 'actualUsage'] = 0
station_data.loc[malicious_users, 'cancellations'] = np.random.randint(3, 5, size=10)

# Encrypt sensitive fields
station_data['encrypted_reservationCount'] = station_data['reservationCount'].apply(lambda x: encrypt(str(x).encode(), key))
station_data['encrypted_actualUsage'] = station_data['actualUsage'].apply(lambda x: encrypt(str(x).encode(), key))
station_data['encrypted_cancellations'] = station_data['cancellations'].apply(lambda x: encrypt(str(x).encode(), key))
station_data['encrypted_chargeTimeHrs'] = station_data['chargeTimeHrs'].apply(lambda x: encrypt(str(x).encode(), key))

# Decrypt data in a secure environment
station_data['decrypted_reservationCount'] = station_data['encrypted_reservationCount'].apply(lambda x: int(decrypt(x, key).decode()))
station_data['decrypted_actualUsage'] = station_data['encrypted_actualUsage'].apply(lambda x: int(decrypt(x, key).decode()))
station_data['decrypted_cancellations'] = station_data['encrypted_cancellations'].apply(lambda x: int(decrypt(x, key).decode()))
station_data['decrypted_chargeTimeHrs'] = station_data['encrypted_chargeTimeHrs'].apply(lambda x: float(decrypt(x, key).decode()))

# Apply Isolation Forest on decrypted data
X = station_data[['decrypted_reservationCount', 'decrypted_actualUsage', 'decrypted_cancellations', 'decrypted_chargeTimeHrs']]
iso_forest = IsolationForest(n_estimators=100, contamination=0.15, random_state=42)
station_data['anomaly'] = iso_forest.fit_predict(X)
station_data['is_malicious'] = station_data['anomaly'] == -1

# Stackelberg Model: Dynamic Pricing based on detected anomalies
base_price = 5  # Base price per hour for charging
premium_multiplier = 1.5  # Premium multiplier for malicious users
station_data['price_per_hour'] = base_price

# Update prices based on malicious behavior
station_data.loc[station_data['is_malicious'], 'price_per_hour'] = base_price * premium_multiplier

# Visualization 1: Comparison of Malicious vs Non-Malicious Users (Key Metrics)
metrics = ['decrypted_reservationCount', 'decrypted_actualUsage', 'decrypted_cancellations', 'decrypted_chargeTimeHrs']
station_data.groupby('is_malicious')[metrics].mean().plot(kind='bar', figsize=(10, 6))
plt.title('Comparison of Malicious vs Non-Malicious Users')
plt.xlabel('Malicious Status (0 = Non-Malicious, 1 = Malicious)')
plt.ylabel('Average Value')
plt.legend(title='Metrics')
plt.show()

# Visualization 2: Charge Time Distribution
plt.figure(figsize=(8, 6))
sns.boxplot(x='is_malicious', y='decrypted_chargeTimeHrs', data=station_data, palette="coolwarm")
plt.title('Charge Time Distribution: Malicious vs Non-Malicious Users')
plt.xlabel('Malicious Status (0 = Non-Malicious, 1 = Malicious)')
plt.ylabel('Charge Time (Hrs)')
plt.show()

# Visualization 3: Reservation Count Distribution
plt.figure(figsize=(8, 6))
sns.boxplot(x='is_malicious', y='decrypted_reservationCount', data=station_data, palette="coolwarm")
plt.title('Reservation Count Distribution: Malicious vs Non-Malicious Users')
plt.xlabel('Malicious Status (0 = Non-Malicious, 1 = Malicious)')
plt.ylabel('Reservation Count')
plt.show()

# Visualization 4: Cancellations Distribution
plt.figure(figsize=(8, 6))
sns.boxplot(x='is_malicious', y='decrypted_cancellations', data=station_data, palette="coolwarm")
plt.title('Cancellations: Malicious vs Non-Malicious Users')
plt.xlabel('Malicious Status (0 = Non-Malicious, 1 = Malicious)')
plt.ylabel('Cancellations')
plt.show()

# Visualization 5: Bar Graph of Malicious vs Non-Malicious Users
malicious_counts = station_data['is_malicious'].value_counts()
plt.figure(figsize=(8, 6))
sns.barplot(x=malicious_counts.index, y=malicious_counts.values, palette="coolwarm")
plt.title('Count of Malicious vs Non-Malicious Users')
plt.xlabel('Malicious Status (0 = Non-Malicious, 1 = Malicious)')
plt.ylabel('Count')
plt.xticks([0, 1], ['Non-Malicious', 'Malicious'])
plt.show()

# Visualization 6: Bar Graph of Dynamic Pricing for Malicious and Non-Malicious Users
price_data = station_data.groupby('is_malicious')['price_per_hour'].mean().reset_index()
plt.figure(figsize=(8, 6))
sns.barplot(x=price_data['is_malicious'], y=price_data['price_per_hour'], palette="coolwarm")
plt.title('Dynamic Pricing: Malicious vs Non-Malicious Users')
plt.xlabel('Malicious Status (0 = Non-Malicious, 1 = Malicious)')
plt.ylabel('Price per Hour ($)')
plt.xticks([0, 1], ['Non-Malicious', 'Malicious'])
plt.show()
